In [ ]:
'''
   Readme first: use pip command to install prerequisite

    Package                Version
    ---------------------- -----------
    aphrodite              0.1.4
    import-ipynb           0.1.3
    numpy                  1.19.4
    pandas                 1.1.4
    poseidon               0.3.1
    SQLAlchemy             1.3.20
    Werkzeug               0.16.1
    enos-mqtt-sdk-python
'''

In [1]:
import datetime
import sys
import pandas as pd
import sqlalchemy
import numpy
import json
from poseidon import poseidon
from datetime import datetime , timedelta
from dateutil.relativedelta import relativedelta

#import commond.ipynb from same folder
import import_ipynb
from conf_kom import *
from common import *

importing Jupyter notebook from conf_kom.ipynb
importing Jupyter notebook from common.ipynb
03 Aug 2020 DAQ FilesPQC887


In [2]:
def getMeasurePoint_21(deviceId, startTime, endTime, interval, measurement_point):
    '''This function calls get the data as per asset and time duration provided. Interval is a mandatory filed. '''
    try:
        urlData = "%s/tsdb-service/v2.1/ai-normalized?orgId=%s" % (address, orgId)
        requestBody = {
            "assetIds": "%s" % deviceId,
            "measurepointsWithLogic": "avg(" + measurement_point + ")",
            "startTime": "%s" % startTime,
            "endTime": "%s" % endTime,
            "interval": "%s" % interval,
            "localTimeAccuracy": "true",
            "localTimeFormat": 0,
            "itemFormat": 2
        }
        req = poseidon.urlopen(appkey, appsecret, urlData, requestBody)
        print(urlData, requestBody )
        if req is None: 
            print(urlData, requestBody )
            return None
        df = pd.DataFrame()
        if req['code'] == 80400:
            value = 'NA'
        elif req['data']['items']:
            if req is not None:
                for item in req['data']['items']:
                    #print(item)
                    value = item['value']
                    d = {'localtime': item['localtime'], measurement_point: item['value']}
                    df = pd.DataFrame(d)
                    return df
            else:
                value = 0
        else:
            value = 0
        return df
    except Exception as err:
        print(err)

In [3]:
def getAssetNodes(treeId):
    try:
        urlAsset = '%s/asset-tree-service/v2.1/asset-nodes?action=searchRelatedAsset&treeId=%s&orgId=%s' % (
            address, treeId, orgId)
        requestBodyAsset = {
            "action": "searchRelatedAsset",
            "projection": ["assetId", "name"]
        }
        req = poseidon.urlopen(appkey, appsecret, urlAsset, requestBodyAsset)
        return req;
    except Exception as err:
        print(err)    

In [4]:
def getMeasurePoint_20(deviceId, startTime, endTime, interval, measurement_point):
    '''This function calls get the data as per asset and time duration provided. Interval is a mandatory filed. '''
    try:
        urlData = "%s/tsdb-service/v2.0/ai-normalized?orgId=%s&assetIds=%s&measurepointsWithLogic=avg(%s)&interval=%s&startTime=%s&endTime=%s&pageSize=2000&localTimeAccuracy=%s" \
        % (address, orgId, deviceId, measurement_point, interval,startTime, endTime, "true")
        req = poseidon.urlopen(appkey, appsecret, urlData)
        
        if req is None: 
            print(urlData )
            return None
        #print(req)
        df = pd.DataFrame()
        if req['msg'] != 'success':
            value = 'NA'
        elif req['data']['items']:
            df = pd.DataFrame(req['data']['items'])
        else:
            value = 0
        return df
    except Exception as err:
        print(err)

In [5]:
def getMeasurePoint_rawdata_20(deviceId, apitype, startTime, endTime, interval, measurement_point):
    '''This function calls get the data as per asset and time duration provided. Interval is a mandatory filed. '''
    try:
        urlData = "%s/tsdb-service/v2.0/%s?orgId=%s&assetIds=%s&measurepoints=%s&interval=%s&startTime=%s&endTime=%s&pageSize=60000&localTimeAccuracy=%s" \
        % (address, apitype, orgId, deviceId, measurement_point, interval,startTime, endTime, "true")
        req = poseidon.urlopen(appkey, appsecret, urlData)
        
        if req is None: 
            print(urlData )
            return None
        #print(req)
        df = pd.DataFrame()
        if req['msg'] != 'success':
            value = 'NA'
        elif req['data']['items']:
            df = pd.DataFrame(req['data']['items'])
        else:
            value = 0
        return df
    except Exception as err:
        print(err)

In [6]:
'''
    Test to getMeasurePoint_rawdata_20
    for Active_Energy, must use generic API
    for Active_Power, must use ai API
'''

res = {}
req = getAssetNodes(treeId)
startTime = "2020-11-11%2000:00:00"
endTime = "2020-11-11%2023:59:59"
mpName = "Active_Energy"
for val in req['data']:
    
    assetId = val['assetId']
    assetName = val['name']['defaultValue']
    
    if assetId in substationIds:
        print(val)
        ds = getMeasurePoint_rawdata_20(assetId, "generic", startTime, endTime, 1, mpName)
        res.update({assetName: ds})
        print(assetId, res)

{'assetId': '2Z86rfOS', 'name': {'i18nValue': {}, 'defaultValue': 'MainSub_03'}}
2Z86rfOS {'MainSub_03':        assetId      timestamp  Active_Energy                localtime
0     2Z86rfOS  1605024005117    8957475.840  2020-11-11 00:00:05.117
1     2Z86rfOS  1605024020390    8957475.840  2020-11-11 00:00:20.390
2     2Z86rfOS  1605024035444    8957475.840  2020-11-11 00:00:35.444
3     2Z86rfOS  1605024050498    8957475.840  2020-11-11 00:00:50.498
4     2Z86rfOS  1605024065568    8957492.224  2020-11-11 00:01:05.568
...        ...            ...            ...                      ...
5727  2Z86rfOS  1605110333438    8969840.640  2020-11-11 23:58:53.438
5728  2Z86rfOS  1605110348492    8969853.952  2020-11-11 23:59:08.492
5729  2Z86rfOS  1605110363562    8969853.952  2020-11-11 23:59:23.562
5730  2Z86rfOS  1605110378616    8969853.952  2020-11-11 23:59:38.616
5731  2Z86rfOS  1605110393966    8969853.952  2020-11-11 23:59:53.966

[5732 rows x 4 columns]}
{'assetId': '5QM4Useo', 'name

In [8]:
''' Sample output
assetId,timestamp,Active_Energy,localtime
2Z86rfOS,1570636802768,3770921.984,2019-10-10 00:00:02.768
2Z86rfOS,1570636822318,3770925.056,2019-10-10 00:00:22.318
2Z86rfOS,1570636843395,3770928.128,2019-10-10 00:00:43.395
2Z86rfOS,1570636863356,3770930.944,2019-10-10 00:01:03.356
2Z86rfOS,1570636883085,3770934.016,2019-10-10 00:01:23.085
2Z86rfOS,1570636903613,3770937.088,2019-10-10 00:01:43.613
2Z86rfOS,1570636924112,3770939.648,2019-10-10 00:02:04.112
2Z86rfOS,1570636943869,3770942.976,2019-10-10 00:02:23.869
2Z86rfOS,1570636964449,3770946.048,2019-10-10 00:02:44.449
2Z86rfOS,1570636985350,3770949.376,2019-10-10 00:03:05.350
2Z86rfOS,1570637006190,3770952.192,2019-10-10 00:03:26.190
2Z86rfOS,1570637026720,3770955.264,2019-10-10 00:03:46.720
2Z86rfOS,1570637047376,3770958.336,2019-10-10 00:04:07.376
2Z86rfOS,1570637066975,3770960.896,2019-10-10 00:04:26.975
2Z86rfOS,1570637087665,3770963.968,2019-10-10 00:04:47.665
2Z86rfOS,1570637108153,3770966.784,2019-10-10 00:05:08.153
'''
def export_measurement_point(assetName, apitype, assetId, beginDate, endDate, mpName, folder_name):    
    ds = pd.DataFrame()    
    date = datetime.strptime(beginDate, '%Y-%m-%d %H:%M:%S')
    delta = datetime.strptime(endDate, '%Y-%m-%d %H:%M:%S') - date
    for i in range(delta.days): 
        dayBefore = date
        print(dayBefore)
        date += relativedelta(days=1)
        print(date)
        if (date > datetime.strptime(endDate, '%Y-%m-%d %H:%M:%S')) == True:
            break

        val = getMeasurePoint_rawdata_20(assetId, apitype, str(dayBefore).replace(" ", "%20"), str(date).replace(" ", "%20"), 1, mpName)
        ds = ds.append(val)

    print(ds.head(5))
    print(ds.shape)

    # export the mp to files by readiable file name
    export_dataset(ds,assetName + '-' + assetId + '-' + mpName, folder_name)
    del ds
    gc.collect()

In [7]:
'''this is the entry point to download the all assets in the assettree defined in conf_kom file'''

beginDate = "2019-10-10 00:00:01"  ## start time of target dataset
endDate = "2020-11-13 17:00:00"    ## end time of target dataset
mpName = "Active_Energy"           ## measurement point name whose TS data will be exported
apiName = "generic"                ## specify which bucket from TSDB to read from.

req = getAssetNodes(treeId)        ## will look through the asset tree to download the dataset

for val in req['data']:
    assetId = val['assetId']
    assetName = val['name']['defaultValue']
    
    if assetId in substationIds: 
        print("Download Mp of {} {}".format( assetName, assetId ))
        export_measurement_point(assetName, apiName , assetId, beginDate, endDate, mpName, folder_name)

Download Mp of MainSub_03 2Z86rfOS
2019-10-10 00:00:01
2019-10-11 00:00:01
2019-10-11 00:00:01
2019-10-12 00:00:01
2019-10-12 00:00:01
2019-10-13 00:00:01
2019-10-13 00:00:01
2019-10-14 00:00:01
2019-10-14 00:00:01
2019-10-15 00:00:01
2019-10-15 00:00:01
2019-10-16 00:00:01
2019-10-16 00:00:01
2019-10-17 00:00:01
2019-10-17 00:00:01
2019-10-18 00:00:01
2019-10-18 00:00:01
2019-10-19 00:00:01
2019-10-19 00:00:01
2019-10-20 00:00:01
2019-10-20 00:00:01
2019-10-21 00:00:01
2019-10-21 00:00:01
2019-10-22 00:00:01
2019-10-22 00:00:01
2019-10-23 00:00:01
2019-10-23 00:00:01
2019-10-24 00:00:01
2019-10-24 00:00:01
2019-10-25 00:00:01
2019-10-25 00:00:01
2019-10-26 00:00:01
2019-10-26 00:00:01
2019-10-27 00:00:01
2019-10-27 00:00:01
2019-10-28 00:00:01
2019-10-28 00:00:01
2019-10-29 00:00:01
2019-10-29 00:00:01
2019-10-30 00:00:01
2019-10-30 00:00:01
2019-10-31 00:00:01
2019-10-31 00:00:01
2019-11-01 00:00:01
2019-11-01 00:00:01
2019-11-02 00:00:01
2019-11-02 00:00:01
2019-11-03 00:00:01
2019-

2020-05-01 00:00:01
2020-05-02 00:00:01
2020-05-02 00:00:01
2020-05-03 00:00:01
2020-05-03 00:00:01
2020-05-04 00:00:01
2020-05-04 00:00:01
2020-05-05 00:00:01
2020-05-05 00:00:01
2020-05-06 00:00:01
2020-05-06 00:00:01
2020-05-07 00:00:01
2020-05-07 00:00:01
2020-05-08 00:00:01
2020-05-08 00:00:01
2020-05-09 00:00:01
2020-05-09 00:00:01
2020-05-10 00:00:01
2020-05-10 00:00:01
2020-05-11 00:00:01
2020-05-11 00:00:01
2020-05-12 00:00:01
2020-05-12 00:00:01
2020-05-13 00:00:01
2020-05-13 00:00:01
2020-05-14 00:00:01
2020-05-14 00:00:01
2020-05-15 00:00:01
2020-05-15 00:00:01
2020-05-16 00:00:01
2020-05-16 00:00:01
2020-05-17 00:00:01
2020-05-17 00:00:01
2020-05-18 00:00:01
2020-05-18 00:00:01
2020-05-19 00:00:01
2020-05-19 00:00:01
2020-05-20 00:00:01
2020-05-20 00:00:01
2020-05-21 00:00:01
2020-05-21 00:00:01
2020-05-22 00:00:01
2020-05-22 00:00:01
2020-05-23 00:00:01
2020-05-23 00:00:01
2020-05-24 00:00:01
2020-05-24 00:00:01
2020-05-25 00:00:01
2020-05-25 00:00:01
2020-05-26 00:00:01


Download Mp of MainSub_02 5QM4Useo
2019-10-10 00:00:01
2019-10-11 00:00:01
2019-10-11 00:00:01
2019-10-12 00:00:01
2019-10-12 00:00:01
2019-10-13 00:00:01
2019-10-13 00:00:01
2019-10-14 00:00:01
2019-10-14 00:00:01
2019-10-15 00:00:01
2019-10-15 00:00:01
2019-10-16 00:00:01
2019-10-16 00:00:01
2019-10-17 00:00:01
2019-10-17 00:00:01
2019-10-18 00:00:01
2019-10-18 00:00:01
2019-10-19 00:00:01
2019-10-19 00:00:01
2019-10-20 00:00:01
2019-10-20 00:00:01
2019-10-21 00:00:01
2019-10-21 00:00:01
2019-10-22 00:00:01
2019-10-22 00:00:01
2019-10-23 00:00:01
2019-10-23 00:00:01
2019-10-24 00:00:01
2019-10-24 00:00:01
2019-10-25 00:00:01
2019-10-25 00:00:01
2019-10-26 00:00:01
2019-10-26 00:00:01
2019-10-27 00:00:01
2019-10-27 00:00:01
2019-10-28 00:00:01
2019-10-28 00:00:01
2019-10-29 00:00:01
2019-10-29 00:00:01
2019-10-30 00:00:01
2019-10-30 00:00:01
2019-10-31 00:00:01
2019-10-31 00:00:01
2019-11-01 00:00:01
2019-11-01 00:00:01
2019-11-02 00:00:01
2019-11-02 00:00:01
2019-11-03 00:00:01
2019-

2020-05-01 00:00:01
2020-05-02 00:00:01
2020-05-02 00:00:01
2020-05-03 00:00:01
2020-05-03 00:00:01
2020-05-04 00:00:01


KeyboardInterrupt: 